In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os

import polars as pl
from freezegun import freeze_time

from src.activities import (
    get_activities_df,
    get_activity_summaries,
    get_day_of_week_summaries,
    get_weekly_summaries,
)
from src.auth_manager import get_strava_client
from src.constants import COACH_ROLE
from src.email_manager import (
    send_email,
    training_week_to_html,
    training_week_update_to_html,
)
from src.supabase_client import (
    get_training_week_with_coaching,
    upsert_training_week_update,
    upsert_training_week_with_coaching,
)
from src.training_week import generate_training_week
from src.training_week_update import get_updated_training_week
from src.types.mid_week_analysis import MidWeekAnalysis


# Sunday Night Training Week Message

In [3]:
@freeze_time("2024-08-11 20:00:00")
def get_activities_df_wrapper(strava_client):
    return get_activities_df(strava_client)

client_preferences = "A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays"
sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {client_preferences}\n"

# activities setup
athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)
activities_df = get_activities_df_wrapper(strava_client)

# gen training week pipeline
day_of_week_summaries = get_day_of_week_summaries(activities_df)
weekly_summaries = get_weekly_summaries(activities_df)
training_week_with_coaching = generate_training_week(
    sysmsg_base=sysmsg_base,
    weekly_summaries=weekly_summaries,
    day_of_week_summaries=day_of_week_summaries,
)

# save data to db and trigger email
upsert_training_week_with_coaching(
    athlete_id=athlete_id, training_week_with_coaching=training_week_with_coaching
)
send_email(
    subject="Training Schedule Just Dropped 🏃",
    html_content=training_week_to_html(training_week_with_coaching),
)

athlete_id='98390356' token still valid until 2024-08-19 02:00:31+00:00


{'message_id': '<202408182143.60797480276@smtp-relay.mailin.fr>',
 'message_ids': None}

# Mid-Week Update

In [5]:
@freeze_time("2024-08-13 20:00:00")
def mock_get_activity_summaries(strava_client, num_weeks=8):
    return get_activity_summaries(strava_client, num_weeks)

client_preferences = "A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays"
sysmsg_base = f"{COACH_ROLE}\nYour client has included the following preferences: {client_preferences}\n"

athlete_id = os.environ["JAMIES_ATHLETE_ID"]
strava_client = get_strava_client(athlete_id)

training_week_with_coaching = get_training_week_with_coaching(athlete_id)
current_weeks_activity_summaries = mock_get_activity_summaries(strava_client, num_weeks=1)
mid_week_analysis = MidWeekAnalysis(
    activities=current_weeks_activity_summaries,
    training_week=training_week_with_coaching.training_week,
)
training_week_update_with_planning = get_updated_training_week(
    sysmsg_base=sysmsg_base, 
    mid_week_analysis=mid_week_analysis
)

upsert_training_week_update(
    athlete_id=athlete_id,
    mid_week_analysis=mid_week_analysis,
    training_week_update_with_planning=training_week_update_with_planning,
)

send_email(
    subject="Training Schedule Update 🏃",
    html_content=training_week_update_to_html(
        mid_week_analysis=mid_week_analysis, 
        training_week_update_with_planning=training_week_update_with_planning
    ),
)

athlete_id='98390356' token still valid until 2024-08-19 02:00:31+00:00


APIResponse(data=[{'id': 15, 'athlete_id': 98390356, 'activities': '[{"date_and_time": "Monday, August 12, 2024 06:06 AM", "distance_in_miles": 8.01, "elevation_gain_in_feet": 183.73, "pace_minutes_per_mile": 9.43}, {"date_and_time": "Tuesday, August 13, 2024 06:15 AM", "distance_in_miles": 4.01, "elevation_gain_in_feet": 52.49, "pace_minutes_per_mile": 10.64}]', 'training_week': '[{"day": "sat", "session_type": "long run", "distance": 15.0, "notes": "Long run at easy pace, aiming for 15 miles to build endurance for the marathon."}, {"day": "wed", "session_type": "speed workout", "distance": 6.0, "notes": "Speed workout, including intervals: 2 miles warm-up, 2x1.5 miles at 10k pace (7m 30s), 1 mile cool-down."}, {"day": "sun", "session_type": "easy run", "distance": 5.0, "notes": "Recovery run at easy pace to help recover from long run."}, {"day": "mon", "session_type": "moderate run", "distance": 5.0, "notes": "Moderate pace run, slightly faster than easy pace."}, {"day": "tues", "ses

In [12]:
from src.lambda_function import lambda_handler

@freeze_time("2024-08-18 20:00:00")
def test_sunday_workflow():
    lambda_handler({}, {})

@freeze_time("2024-08-13 20:00:00")
def test_mid_week_workflow():
    lambda_handler({}, {})

test_sunday_workflow()
test_mid_week_workflow()

athlete_id='98390356' token still valid until 2024-08-19 02:00:31+00:00
athlete_id='98390356' token still valid until 2024-08-19 02:00:31+00:00
